In [48]:
import pandas as pd
import matplotlib.pyplot as plt
from circuit_class import Circuit
from standingwave_class import Standingwave
from circuit_class import get_test_name
import gc
import sys
import numpy as np
from tqdm import tqdm_notebook as tqdm
import os
from platypus import NSGAII, Problem, Real, nondominated, Integer
import matplotlib.pyplot as plt
from platypus.problems import DTLZ2
import scipy

# 並列共振回路ver.

シミュレーションを回しなおさなくていい場合、**<span style="color: green; "> 緑太字で示した部分</span>**をコメントアウトする

## コイルのインダクタンスLと直列負荷抵抗Rを変数としたシミュレーション

### <span style="color: red; ">変数指定</span>
テストするL, Rの範囲と間隔を指定(単位はLはμH, RはΩ)

In [49]:
# Lは単位μH, 範囲指定はlogスケールで行う
L_start = -1
L_end = 3
L_num = 60

# Rは単位Ω, 範囲指定は線形スケールで行う
R_start = 0
R_end = 3
R_num = 60

# 結合係数Kは0.1で固定
K=0.1

# テスト元にするcirファイルのパス
cir_path = '/Users/yuki/Documents/LTspice/test/parallel_0912/main_parallel'

変数のリストを用意

In [50]:
# 浮動少数点数型の変数の配列
L_values_num = list(np.logspace(L_start, L_end, L_num))
R_values_num = list(np.logspace(R_start, R_end, R_num))

# 文字列型の変数の配列
L_values = list(map(lambda x: str(x)+"u", L_values_num))
R_values = list(map(lambda x: str(x), R_values_num))

収集するデータのDataFrame用のカラムを準備

In [57]:
columns = ['L','L_str', 'R', 'R_str', 'V(out)', 'W']

## <span style="color: red; ">条件増やしたり変えたらここも変更チェック
条件を表す文字列を作成

In [58]:
condition = 'parallel_0912'
condition += f'_L:{L_values[0]}~{L_values[-1]}(n={L_num})_R:{R_values[0]}~{R_values[-1]}(n={R_num})_K:{K}'
condition

'parallel_0912_L:0.1u~1000.0u(n=60)_R:1.0~1000.0(n=60)_K:0.1'

結果保存用の条件別のフォルダを作成

In [59]:
result_dir = f'result/{condition}'
if not os.path.isdir(result_dir):
    os.mkdir(result_dir)

## 全パターンのシミュレーションの実行

### <span style="color: green; ">完了している場合はコメントアウトしておく

In [60]:
# change_elements = {}
# change_elements['K'] = K

# for L in tqdm(L_values):
#     parallel = Circuit('/Users/yuki/Documents/LTspice/test/parallel_0912/main_parallel')
#     change_elements['L'] = L
#     for R in tqdm(R_values):
#         change_elements['R'] = R
#         parallel.change_element_run(change_elements)
#     del parallel
#     gc.collect()

# L別にRと出力側電圧振幅の関係をみる

キーがL, valueがRごとの振幅のリスト、となる辞書型配列L_amp_dictを取得

In [ ]:
change_elements = {}
change_elements['K'] = K

L_amp_dict = {}
for L in tqdm(L_values):
    parallel = Standingwave(cir_path)
    change_elements['L'] = L
    for R in R_values:
        change_elements['R'] = R
        parallel.change_element_df_save(change_elements)
    parallel.t_cut(0.0049)
    L_amp_dict[L] = list(parallel.get_amp_means()["V(test)"].values())
    del parallel
    gc.collect()

波形確認

In [ ]:
# change_elements = {}
# change_elements['K'] = K

# # count = 0

# for L in tqdm(L_values[50:60]):
#     parallel = Standingwave(cir_path)
#     change_elements['L'] = L
#     for R in R_values:
#         change_elements['R'] = R
#         parallel.change_element_df_save(change_elements)
#     parallel.t_cut(0.0049)
# #     if count>55:
#     parallel.dfs["V(test)"][f'K=0.1L={L}R=1000.0'].plot(x="t", y="V(test)", title = f'L={L}H')
#     del parallel
#     gc.collect()

描画(x, yどちらも対数軸)して保存

In [ ]:
ncols=3
nrows = 1+len(L_values)//ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
plt_count = 0
for L in L_values:
    R_amp_df = pd.DataFrame({"R":R_values_num, "V(out)":L_amp_dict[L]})
    R_amp_df.plot(x="R", y="V(out)", title = f'L={L}H', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
    plt_count+=1
plt.savefig(f'{result_dir}/R_amp.png')
plt.close('all')

# R別にLと出力側電圧振幅の関係をみる

R_amp_dictは{R:[Lごとの電圧のリスト]]

In [ ]:
R_amp_dict = {}
for i in tqdm(range(len(R_values))):
    temp = []
    for L in L_values:
        temp.append(L_amp_dict[L][i])
    R_amp_dict[R_values[i]] = temp 

In [ ]:
# R_amp_dict

描画(x, yどちらも対数軸)して保存

In [ ]:
ncols=3
nrows = 1+len(R_values)//ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
plt_count = 0
for R in R_values:
    L_amp_df = pd.DataFrame({"L":L_values_num, "V(out)":R_amp_dict[R]})
    L_amp_df.plot(x="L", y="V(out)", title = f'R={R}Ω', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
    plt_count+=1
plt.savefig(f'{result_dir}/L_amp.png')
plt.close('all')

### 出力電圧が最高だったテストパターン

In [ ]:
max_L = ''
max_R = ''
max_test = ''
max_V = 0.0
for L in L_values:
    for i in range(len(R_values)):
        v = L_amp_dict[L][i]
        if v>max_V:
            max_L = L
            max_R = R_values[i]
            max_V = v
print(f"L={max_L}H\nR={max_R}Ω\n→V = {max_V}")

# L別でRと整定後の平均消費電力の関係をみる

In [ ]:
change_elements = {}
change_elements['K'] = K

L_W_dict = {}
count = 0
for L in tqdm(L_values):
    parallel = Standingwave(cir_path)
    change_elements['L'] = L
    W_temp = []
    for R in R_values:
        change_elements['R'] = R
        W_mean = parallel.get_W('I(R1)', 'V(a)', change_elements=change_elements)['W'].mean()
        W_temp.append(W_mean)
    L_W_dict[L] = W_temp
    del parallel
    gc.collect()
    count+=1
    sys.stdout.write("\r{}/{}完了".format(count, len(L_values)))
    sys.stdout.flush()

In [ ]:
ncols=3
nrows = 1+len(L_values)//ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
plt_count = 0
for L in L_values:
    R_W_df = pd.DataFrame({"R":R_values_num, "W":L_W_dict[L]})
    R_W_df.plot(x="R", y="W", title = f'L={L}H', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
    plt_count+=1
plt.savefig(f'{result_dir}/R_W.png')
plt.close('all')

波形確認

In [ ]:
# change_elements = {}
# change_elements['K'] = K

# ncols=3
# nrows = 1+20//ncols

# fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
# plt_count = 0

# for L in tqdm(L_values[59:60]):
#     parallel = Standingwave(cir_path)
#     change_elements['L'] = L
#     hoge = 0
#     for R in R_values:
#         change_elements['R'] = R
#         if hoge < 20:
#             w = parallel.get_W('I(R1)', 'V(a)', change_elements=change_elements)
#             w.plot(x="t", y="W", title = f'R={R}Ω', ax=axes[plt_count//ncols, plt_count%ncols])
#             plt_count+=1
#         hoge +=1
#     del parallel
#     gc.collect()
# plt.savefig(f'{result_dir}/W_waves_L=1000.png')
# plt.close('all')

### 最大消費電力でみる

In [ ]:
# change_elements = {}
# change_elements['K'] = K

# L_Wmax_dict = {}
# count = 0
# for L in tqdm(L_values):
#     parallel = Standingwave(cir_path)
#     change_elements['L'] = L
#     W_temp = []
#     for R in R_values:
#         change_elements['R'] = R
#         W_max = parallel.get_W('I(R1)', 'V(a)', change_elements=change_elements)['W'].max()
#         W_temp.append(W_max)
#     L_Wmax_dict[L] = W_temp
#     del parallel
#     gc.collect()
#     count+=1
#     sys.stdout.write("\r{}/{}完了".format(count, len(L_values)))
#     sys.stdout.flush()

In [ ]:
# ncols=3
# nrows = 1+len(L_values)//ncols

# fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
# plt_count = 0
# for L in L_values:
#     R_Wmax_df = pd.DataFrame({"R":R_values_num, "W(max)":L_Wmax_dict[L]})
#     R_Wmax_df.plot(x="R", y="W(max)", title = f'L={L}H', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
#     plt_count+=1
# plt.savefig(f'{result_dir}/R_W(max).png')
# plt.close('all')

# R別でLと整定後の平均消費電力の関係をみる

In [ ]:
R_W_dict = {}
for i in tqdm(range(len(R_values))):
    temp = []
    for L in L_values:
        temp.append(L_W_dict[L][i])
    R_W_dict[R_values[i]] = temp 

In [ ]:
ncols=3
nrows = 1+len(R_values)//ncols

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
plt_count = 0
for R in R_values:
    L_W_df = pd.DataFrame({"L":L_values_num, "W":R_W_dict[R]})
    L_W_df.plot(x="L", y="W", title = f'R={R}Ω', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
    plt_count+=1
plt.savefig(f'{result_dir}/L_W.png')
plt.close('all')

### 最大消費電力でみる

In [ ]:
# R_Wmax_dict = {}
# for i in tqdm(range(len(R_values))):
#     temp = []
#     for L in L_values:
#         temp.append(L_Wmax_dict[L][i])
#     R_Wmax_dict[R_values[i]] = temp 

In [ ]:
# ncols=3
# nrows = 1+len(R_values)//ncols

# fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(18, 6*nrows))
# plt_count = 0
# for R in R_values:
#     L_Wmax_df = pd.DataFrame({"L":L_values_num, "W(max)":R_Wmax_dict[R]})
#     L_Wmax_df.plot(x="L", y="W(max)", title = f'R={R}Ω', loglog = True, ax=axes[plt_count//ncols, plt_count%ncols])
#     plt_count+=1
# plt.savefig(f'{result_dir}/L_W(max).png')
# plt.close('all')

### 平均消費電力が最小だったテストパターン

In [ ]:
min_L = ''
min_R = ''
min_W = 100000
for L in L_values:
    for i in range(len(R_values)):
        w = L_W_dict[L][i]
        if w<min_W:
            min_L = L
            min_R = R_values[i]
            min_W = w
print(f"L={min_L}H\nR={min_R}Ω\n→minW = {min_W}")

# 出力電圧と平均消費電力を２軸プロットする

In [ ]:
columns = ['L', 'R', 'V(out)', 'W']
amp_w_df = pd.DataFrame(columns= columns)

for L in L_values:
    for i in range(len(R_values)):
        amp_w_df = amp_w_df.append({'L': L, 'R': R_values[i], 'V(out)': L_amp_dict[L][i], 'W': L_W_dict[L][i]}, ignore_index=True)

In [ ]:
amp_w_df

In [ ]:
amp_w_df.plot(x="V(out)", y="W", kind='scatter')
plt.savefig(f'{result_dir}/vout_w_scatterd.png')
plt.close('all')

### NSGA-Ⅱを用いたパレート最適解の抽出
多目的最適化問題に用いられる多目的遺伝的アルゴリズムのNSGA-Ⅱ(詳細はhttps://qiita.com/DS27/items/025a52b26a9f2471e67c )を用いて、非劣解を抽出する.
用いているplatypusライブラリに関しては https://qiita.com/pontyo4/items/d8812e4f46850d746fcd を軽く参照

In [ ]:
def obj_func(args):
    L_num = args[0]
    R_num = args[1]
    data = amp_w_df[(amp_w_df['L']==L_values[L_num]) & (amp_w_df['R']==R_values[R_num])]
    return [float(data['V(out)']),float(data.W)]

problem = Problem(2, 2)
problem.directions[:] = [Problem.MAXIMIZE, Problem.MINIMIZE]
L_num = Integer(0, len(L_values)-1)
R_num = Integer(0, len(R_values)-1)
problem.types[:] = [L_num, R_num]
problem.function = obj_func

algorithm = NSGAII(problem, population_size = 50)
algorithm.run(10000)

# 非劣解をとりだす
nondominated_solutions = nondominated(algorithm.result)

# グラフを描画
plt.scatter([s.objectives[0] for s in nondominated_solutions if s.feasible],
           [s.objectives[1] for s in nondominated_solutions if s.feasible])
plt.savefig(f'{result_dir}/vout_w_pareto.png')
plt.close('all')

In [ ]:
df = pd.DataFrame(columns=("L", "R", "V(out)", "W"))
for i in range(len(nondominated_solutions)):
    df.loc[i, "L"] = L_values[int1.decode(nondominated_solutions[i].variables[0])]
    df.loc[i, "R"] = R_values[int2.decode(nondominated_solutions[i].variables[1])]
    df.loc[i, "V(out)"] = nondominated_solutions[i].objectives[0]
    df.loc[i, "W"] = nondominated_solutions[i].objectives[1]
df.sort_values(['L', 'R'], ascending = [False, False])
df.to_csv(result_dir+'/pareto_set.csv')

In [ ]:
df

# 理論式を用いたインダクタンスの算出

矩形コイルのインダクタンスの理論式
$$
L(nH) = 1.62\times10^{-3}\times{d_{out}}^{-1.21}w^{-0.147}d_{avg}^{2.40}n^{1.78}s^{-0.030}
$$
に従って算出する。
なお、各変数の単位はL(nH), Dout, w, Davg, sはそれぞれumである。

In [ ]:
n=125

Dout=30*(10**(3))
w=0.06*(10**(3))
s=0.06*(10**(3))
Din=Dout-w*(2*n)-s*(2*n-1) # ここは図的にこうなる
Davg=(Dout+Din)/2
nmax = (Dout+s)/(2*(s+w))

In [ ]:
nmax
Din

In [ ]:
1.62*(10**(-3))*(Dout**(-1.21))*(w**(-0.147))*(Davg**2.4)*(n**1.78)*(s**(-0.03))

In [3]:
# coding: utf-8                                                                                                                                                                               

import math
from scipy.optimize import newton_krylov

def F(x):
    return [
        -2 * x[0] + x[2]*(1-math.tanh(x[0])**2),
         -2 * x[1] + x[2]*(1-math.tanh(x[1])**2),
         math.tanh(x[0]) + math.tanh(x[1]) - 1
         ]

guess = [1,1,0]

sol = newton_krylov(F, guess, method='lgmres')

print(sol)

[0.54930577 0.54930577 1.46481399]
